In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dataclasses import dataclass, field
import re

In [2]:
data = pd.read_csv('./저장된 데이터.csv')
data.head()

,일시 Run 1,시간 (s) Run 1,힘 (N) Run 1,위치 (m) Run 1,속도 (m/s) Run 1,"가속도, 파랑 (m/s²) Run 1",net force (N) Run 1,acceleration (m/s²) Run 1,일시 실행 2,시간 (s) 실행 2,...,net force (N) 실행 50,acceleration (m/s²) 실행 50,일시 실행 51,시간 (s) 실행 51,힘 (N) 실행 51,위치 (m) 실행 51,속도 (m/s) 실행 51,"가속도, 파랑 (m/s²) 실행 51",net force (N) 실행 51,acceleration (m/s²) 실행 51
0,2023/09/18 12:17:34,0.000,-0.36,0.0,NaN,NaN,NaN,NaN,2023/09/18 12:17:41,0.000,...,NaN,NaN,2023/09/18 12:34:10,0.000,-0.51,0.0,NaN,NaN,NaN,NaN
1,2023/09/18 12:17:34,0.025,-0.36,0.0,0.000000,NaN,NaN,NaN,2023/09/18 12:17:41,0.025,...,NaN,NaN,2023/09/18 12:34:10,0.025,-0.52,0.0,0.0,NaN,NaN,NaN
2,2023/09/18 12:17:34,0.050,-0.37,0.0,0.000000,0.016,NaN,NaN,2023/09/18 12:17:41,0.050,...,NaN,NaN,2023/09/18 12:34:10,0.050,-0.51,0.0,0.0,0.0,NaN,NaN
3,2023/09/18 12:17:34,0.075,-0.36,0.0,0.000808,0.097,NaN,NaN,2023/09/18 12:17:41,0.075,...,NaN,NaN,2023/09/18 12:34:10,0.075,-0.51,0.0,0.0,0.0,NaN,NaN
4,2023/09/18 12:17:34,0.100,-0.36,0.0,0.004000,0.199,NaN,NaN,2023/09/18 12:17:41,0.100,...,NaN,NaN,2023/09/18 12:34:10,0.100,-0.51,0.0,0.0,0.0,NaN,NaN


In [27]:
@dataclass
class Run:
    id: int
    t: pd.Series = field(default_factory=pd.Series)
    F: pd.Series = field(default_factory=pd.Series)
    x: pd.Series = field(default_factory=pd.Series)
    v: pd.Series = field(default_factory=pd.Series)
    a: pd.Series = field(default_factory=pd.Series)

class Runs:
    def __init__(self, path: str) -> None:
        data = pd.read_csv(path).dropna(axis=0, how='all')

        self.runs = {}
        for column in data.columns:
            matched = re.match(r'([a-zA-Zㄱ-힣, ]+) (\(.*\)) (?:Run|실행) (\d+)', column)

            if not matched:
                continue

            name = matched.group(1)
            units = matched.group(2)
            run_id = int(matched.group(3))

            if run_id not in self.runs:
                self.runs[run_id] = Run(run_id)
            
            match name:
                case '시간':
                    self.runs[run_id].t = data[column].to_numpy()
                case '힘':
                    self.runs[run_id].F = data[column].to_numpy()
                case '위치':
                    self.runs[run_id].x = data[column].to_numpy()
                case '속도':
                    self.runs[run_id].v = data[column].to_numpy()
                case '가속도, 파랑':
                    self.runs[run_id].a = data[column].to_numpy()
                case _:
                    pass

    def __getitem__(self, key: int) -> Run:
        return self.runs[key]

In [31]:
runs = Runs('./저장된 데이터.csv')
# plt.plot(runs[1].t, runs[1].v)
r1 = pd.DataFrame({
    't': runs[1].t,
    'F': runs[1].F,
    'x': runs[1].x,
    'v': runs[1].v,
    'a': runs[1].a
})

# 일시랑 net 어쩌구에는 nan이 없어서 제거가 안됨
r1.tail(n=10)

,t,F,x,v,a
174,NaN,NaN,NaN,NaN,NaN
175,NaN,NaN,NaN,NaN,NaN
176,NaN,NaN,NaN,NaN,NaN
177,NaN,NaN,NaN,NaN,NaN
178,NaN,NaN,NaN,NaN,NaN
179,NaN,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,NaN,NaN
181,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN
183,NaN,NaN,NaN,NaN,NaN
